In [1]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import seaborn as sns
from mesa.space import NetworkGrid
import networkx as nx
import os, sys
from itertools import product
from matplotlib.animation import FuncAnimation
#plt.style.use('seaborn-pastel')
#from tqdm.autonotebook import tqdm

In [11]:
class MoneyAgent(Agent):
    
    #agents = []

    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.m = self.model.T
        self.lamda = round(random.uniform(0.1,1),1)
        if (self.lamda == 1):
            while (self.lamda == 1):
                self.lamda = round(random.uniform(0.1,1),1)
        #self.agents.append(self)
            
    def give_money(self):
        neighbors_nodes = self.model.grid.get_neighbors(self.pos, include_center=False)
        neighbors = self.model.grid.get_cell_list_contents(neighbors_nodes)
        
       
        #neighbors_nodes = list(nx.all_neighbors(self.model.G,self.unique_id))
        #neighbors = []
        #for node in neighbors_nodes:
        #    for agent in self.agents:
        #        if(agent.unique_id == node):
        #            neighbors.append(agent)
                    
        
        epsilon = random.random()
        if len(neighbors) > 1 :
            other = self.random.choice(neighbors)
            #w = self.model.G[self.unique_id][other.unique_id]['weight']
            #print(w)
            if(other.unique_id != self.unique_id): 
                xi = self.m
                xj = other.m
                delta_m = (1-self.lamda)*(xi - epsilon*(xi + xj))
                xi_new = xi - delta_m
                xj_new = xj + delta_m
                other.m = xj_new
                self.m = xi_new
                self.model.trade+=1
                
    def move(self):
        possible_steps = [node for node in self.model.grid.get_neighbors(self.pos, include_center=False)
                          if self.model.grid.is_cell_empty(node)]
        if len(possible_steps) > 0:
            new_position = self.random.choice(possible_steps)
            self.model.grid.move_agent(self, new_position)
                
    def Local_Attachment(self):
        node1 = random.choice(self.model.nodes)
        node2 = random.choice(self.model.nodes)
        while(self.model.G.has_edge(node1,node2)==True):
            node2 = random.choice(self.model.nodes)
            
        #adding nodes 1 and 2 to NetworkGrid if they aren't present already
        if(self.model.grid.is_cell_empty(node2)==True):
            a = MoneyAgent(node2,self.model)
            self.model.grid.place_agent(a,node2)
        if(self.model.grid.is_cell_empty(node1)==True):
            a = MoneyAgent(node1,self.model)
            self.model.grid.place_agent(a,node1)
        self.model.G.add_edge(node1,node2,weight = round(random.uniform(0.1,1),1))
        
    
    def Link_Deletion(self):
        node1 = random.sample(self.model.G.nodes,1)[0]
        node2 = random.sample(self.model.G.nodes,1)[0]
        while(self.model.G.has_edge(node1,node2)==False):
            node2 = random.sample(self.model.G.nodes,1)[0]
        self.model.G.remove_edge(node1,node2)
        self.move() #come back again here (?)
        
        
                 
    def step(self):
        if self.m > 0:
            self.give_money()
        self.Local_Attachment()
        self.Link_Deletion()
        self.move()

In [12]:
class BoltzmannWealthModelNetwork(Model):
    """A model with some number of agents."""

    def __init__(self,T,N): #N- number of agents

        self.N = N
        self.T = T
        self.trade = 0
        
        self.G = nx.Graph()
        self.nodes = np.linspace(1,N,N, dtype = 'int') #creating N random nodes
        self.G.add_nodes_from(self.nodes)
        no = int(N/2)
        nodes1 = random.sample(self.G.nodes,no)
        nodes2 = random.sample(self.G.nodes, int(no/2+1)) #creating random edges and giving it a weight between 0 and 1
        self.G.add_weighted_edges_from((a,b,round(random.uniform(0.1,1),1)) for a,b in product(nodes1, nodes2) if a != b)
        self.grid = NetworkGrid(self.G)
        
        
        self.schedule = RandomActivation(self)
        self.datacollector = DataCollector(model_reporters = {'trade': 'trade'},agent_reporters={"mi":'m','lamda':'lamda' })
        
        for i, node in enumerate(self.G.nodes()):
            a = MoneyAgent(i+1, self)
            self.schedule.add(a)
            # Add the agent to a random node
            self.grid.place_agent(a, node)
            
        self.running = True
        self.datacollector.collect(self)
        
    def Global_Attachment(self):
        node1 = random.choice(self.nodes)
        
        #required if using NetworkGrid
        
        #while(self.G.has_node(node1)==True):
        #    node1 = random.choice(self.nodes)
        #node2 = random.choice(self.nodes)
        #while(self.G.has_node(node2)==True):
        #    node2 = random.choice(self.nodes)
    
        node2 = random.choice(self.nodes)
        while(self.G.has_edge(node1,node2)==True):
            node2 = random.choice(self.nodes)
          
        #adding nodes 1 and 2 to NetworkGrid if they aren't present already
        if(self.grid.is_cell_empty(node2)==True):
            a = MoneyAgent(node2,self)
            self.grid.place_agent(a,node2)
        if(self.grid.is_cell_empty(node1)==True):
            a = MoneyAgent(node1,self)
            self.grid.place_agent(a,node1)
            
        #adding the edge node1-node2
        self.G.add_edge(node1,node2,weight = round(random.uniform(0.1,1),1))
        
        
    #def animate(i):
        #ax = plt.gca()
        #ax.clear() #to clear the previous frame
        #return(nx.draw(self.G))
            
        

    def step(self):
        self.schedule.step()
        # collect data
        self.datacollector.collect(self)

    def run_model(self, n):
        for i in tqdm(range(n)):
            self.step()
            self.trade = 0
            self.Global_Attachment()
              


In [ ]:
N = 500
T = 100
steps = 30
#fig = plt.figure()
model = BoltzmannWealthModelNetwork(T,N)
#ani= FuncAnimation(fig,self.animate,frames = 200, interval=20) #animate function
#plt.show()
model.run_model(steps)
model_df = model.datacollector.get_model_vars_dataframe()
agent_df = model.datacollector.get_agent_vars_dataframe()
agent_df.reset_index(level=1, inplace = True)
agent_df['mt'] = agent_df.mi/T

C:\Users\Public\Anaconda3\lib\site-packages\ipykernel_launcher.py:72: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
